In [1]:
import os
from glob import glob
from zipfile import ZipFile
from natsort import natsorted
from Config import PERIG_CLASSES
from huggingface_hub import snapshot_download
from Source.Utils import create_dir, show_sample_pair
from Source.Trainer import MultiSegmentationTrainer

e:\Github\ImageSegmentation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
layout_dataset = "BDRC/LayoutSegmentation_Dataset"

dataset_path = snapshot_download(
            repo_id=f"{layout_dataset}",
            repo_type="dataset",
            cache_dir="Datasets")

with ZipFile(f"{dataset_path}/data.zip", 'r') as zip:
    zip.extractall(f"{dataset_path}")

print(f"downloaded and extracted the dataset to: {dataset_path}")

Fetching 3 files: 100%|██████████| 3/3 [00:00<?, ?it/s]


downloaded and extracted the dataset to: Datasets\datasets--BDRC--LayoutSegmentation_Dataset\snapshots\88ed979b705f49beacf8acc1321c7e8ba138aa9a


In [3]:
train_data = os.path.join(dataset_path, "train")
val_data = os.path.join(dataset_path, "val")
test_data = os.path.join(dataset_path, "test")

train_x = natsorted(glob(f"{train_data}/images/*.jpg"))
train_y = natsorted(glob(f"{train_data}/masks/*.png"))

valid_x = natsorted(glob(f"{val_data}/images/*.jpg"))
valid_y = natsorted(glob(f"{val_data}/masks/*.png"))

test_x = natsorted(glob(f"{test_data}/images/*.jpg"))
test_y = natsorted(glob(f"{test_data}/masks/*.png"))

print(f"Training data => Images: {len(train_x)}, Masks: {len(train_y)}")
print(f"Validation data => Images: {len(valid_x)}, Masks: {len(valid_y)}")
print(f"Test data => Images: {len(test_x)}, Masks: {len(test_y)}")

Training data => Images: 0, Masks: 0
Validation data => Images: 0, Masks: 0
Test data => Images: 0, Masks: 0


In [4]:
patch_size = 512
batch_size = 32

In [ ]:
output_dir = os.path.join(dataset_path, "Output")
create_dir(output_dir)

segmentation_trainer = MultiSegmentationTrainer(
    train_x,
    train_y,
    valid_x,
    valid_y,
    test_x,
    test_y,
    image_width=patch_size,
    image_height=patch_size,
    batch_size=batch_size,
    network="deeplab",
    output_path=output_dir,
    classes=PERIG_CLASSES)

Initializing Mutliclass Segmentation trainer...


e:\Github\ImageSegmentation#\Source\Trainer.py:474: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [ ]:
epochs = 12
segmentation_trainer.train(epochs=epochs)